# Partie II - Implémentation de Pytorch
Le but de cette partie est d'implémenter Pytorch dans des fonctions et classes utilitaires, pour préparer à l'entrainement des modèles que nous allons tester.


## Installation des librairies et import

#### Installation

In [1]:
# Installation des librairies nécessaires :
! pip install pandas
! pip install matplotlib
! pip install pillow
! pip install tqdm
! pip install import-ipynb
! pip install torch
! pip install torchvision

## Définition des classes pour l'entrainement
Les classes et méthodes définies plus bas sont indépendantes d'un modèle en particulier.

### Classe ImageProcessor (redimensionnement et traitement des images)

On redimensionne les images pour une question de gestion de la mémoire. Les images de base pouvant peser jusqu'à plusieurs centaines de Mo, pytorch peut planter lorsqu'il tente de les transformer en tenseurs.

On les redimensionne donc à la taille :
- 256*256
- 500*500

*Deux tailles différentes pour tester l'impact du nombre de pixels sur les performances du modèle.*

In [1]:
# Nos images sont stockées dans un dossier images/images
# Ces images pèsent bien trop lourd pour être transformées en tenseurs et être traitées par le modèle
# On va donc les réduire à une taille de 256x256 pixels
# On va aussi les normaliser en divisant les valeurs des pixels par 255
# On sauvegarde les images dans un dossier images/images_256
# On crée une fonction qui prend en argument un dataframe et qui sauvegarde les images dans un dossier images/images_256
# On souhaite aussi que la fonction aie une barre de progression

# chemin vers les images du dataset
dataset_path = '../../DATASET/'

from tqdm import tqdm
import pandas as pd
import os
from PIL import Image

def resize_images(df, set_name,size):
    # On crée un dossier images/images_256 s'il n'existe pas déjà
    if not os.path.exists(dataset_path + 'images_'+str(size)+'_'+set_name):
        os.makedirs(dataset_path + 'images_'+str(size)+'_'+set_name)
    
    # On affiche la barre de progression sous forme de pourcentage sur le notebook
    pbar = tqdm(total=len(df), position=0, leave=True)

    # On parcourt le dataframe
    for index, row in df.iterrows():
        # On ouvre l'image
        img = Image.open(dataset_path + 'images/' + row['img_name'])
        # On redimensionne l'image
        img = img.resize((size,size))
        # On augmente le contraste de l'image
        img = img.point(lambda x: x*1.5)

        # On sauvegarde l'image
        img.save(dataset_path + 'images_'+str(size)+'_'+set_name+'/' + row['img_name'])
        # On met à jour la barre de progression
        pbar.update(1)

    print('Images redimensionnées et sauvegardées dans : ' + dataset_path + 'images_'+str(size)+'_'+set_name)
    # On ferme la barre de progression
    pbar.close()
    



In [3]:
# On transforme les images en tenseurs avec pytorch
import torch
from torchvision import transforms
from PIL import Image

# On crée une fonction qui prend en argument le chemin vers une image et qui retourne un tenseur
def img_to_tensor(img_path):
    # On ouvre l'image
    img = Image.open(img_path)
    # On transforme l'image en tenseur
    img_tensor = transforms.ToTensor()(img)
    # On retourne le tenseur
    return img_tensor

# On crée une fonction qui prend en argument un dataframe et qui retourne un dictionnaire 
# contenant le nom du fichier image et le tenseur correspondant
def get_img_tensor_dict(df,set_name,size):
    # On crée un dictionnaire vide
    img_tensor_dict = {}
    # On parcourt le dataframe
    for index, row in df.iterrows():
        # On crée le chemin vers l'image
        img_path = dataset_path +'images_'+str(size)+'_'+set_name+'/'+ row['img_name']
        # On transforme l'image en tenseur
        img_tensor = img_to_tensor(img_path)
        # On ajoute le nom du fichier image et le tenseur correspondant au dictionnaire
        img_tensor_dict[row['img_name']] = img_tensor
    # On retourne le dictionnaire
    print('Dictionnaire créé pour le set '+set_name+ ' de taille : '+str(size)+'x'+str(size))
    print('Nombre d\'images dans le set : '+str(len(img_tensor_dict)))

    return img_tensor_dict





In [ ]:
# On implémente les fonctions de traitement d'image dans une classe ImageProcessor

class ImageProcessor:
    def __init__(self):
        pass

    def generate_img_tensors(self,df,set_name,size):
        tensors = get_img_tensor_dict(df,set_name,size)
        return tensors
    
    def resize_images(self,df,set_name,size):
        resize_images(df, set_name,size)

 

### Classe LabelMap
La classe LabelMap permet d'assigner automatique à chaque super-classe de style architectural son propre tenseur, pour la rendre exploitable en tant que label par le modèle.

In [ ]:
# On créé une fonction qui mappe les labels aux noms des styles architecturaux
def create_label_map(df):
    print('No found label mapping, creating a new one...')
    unique_labels = df.iloc[:, 1].unique()
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    print("Label mapping is the following :")
    display(label_map)
    # On sauvegarde le label_map dans un fichier csv
    df_label_map = pd.DataFrame.from_dict(label_map, orient='index', columns=['label'])
    df_label_map.to_csv('../dataset_doc/csv/label_map.csv')
    return label_map

In [ ]:
# On créé une fonction pour lire un label_map à partir d'un fichier csv



def read_label_map(label_map_path):
    print('Found label mapping, reading it...')
    label_map = pd.read_csv(label_map_path, index_col=0)
    label_map = label_map.to_dict()['label']
    print("Label mapping is the following :")
    display(label_map)
    return label_map

In [ ]:
# On créé une classe LabelMap
class LabelMap:
    def __init__(self):
        pass

    def create_label_map(self,df):
        label_map = create_label_map(df)
        return label_map
    
    def read_label_map(self,label_map_path):
        label_map = read_label_map(label_map_path)
        return label_map

### Classe CustomDataset

La classe CustomDataset implémente le module Dataset de Pytorch.
Elle permet de faire passer à un modèle les tenseurs d'images ainsi que les labels.\
Elle permet aussi de mapper les labels (c'est à dire associer un tenseur à chaque label afin de le rendre "ingérable" par le modèle).

In [ ]:
# On créé un dataset custom contenant les tenseurs des images et les labels
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, img_tensor_dict, df, label_map):
        self.img_tensor_dict = img_tensor_dict
        self.df = df
        self.label_map = label_map

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # On récupère le nom du fichier image
        img_name = self.df['img_name'][index]
        # On récupère le tenseur correspondant
        img_tensor = self.img_tensor_dict[img_name]
        # On récupère le label
        label_str = self.df['parent_id'][index]
        label = self.label_map[label_str]
        # On retourne le tenseur et le label
        return img_tensor, label
    
    

### Classe Trainer (et fonction d'entrainement)

In [ ]:
# On créé une fonction d'entrainement du modèle
# Cette fonction possède aussi une barre de progression sous forme de pourcentage avec tqdm
# Cette barre de progression se rempli à chaque epoch

def train_model(model, dataloader, epochs, optimizer, criterion):

    # On parcourt les epochs
    for epoch in range(epochs):

       

        # On affiche l'epoch en cours
        print(f'epoch {epoch+1}/{epochs}')

        # On affiche la barre de progression sous forme de pourcentage sur le notebook
        pbar = tqdm(total=len(dataloader), position=0, leave=True)

        

        # On initialise la running_loss à 0
        running_loss = 0.0

        # On parcourt les batches du dataloader
        for i, data in enumerate(dataloader, 0):

            # On affiche la barre de progression sous forme de pourcentage sur le notebook
            pbar.update(1)

            # On récupère les tenseurs et les labels
            inputs, labels = data

            # On met les gradients à zéro
            optimizer.zero_grad()

            # On fait une prédiction
            outputs = model(inputs)

            # On calcule la fonction de coût
            loss = criterion(outputs, labels)

            # On calcule les gradients
            loss.backward()

            # On met à jour les paramètres
            optimizer.step()

            # Tous les 20 batches, on affiche :
            running_loss += loss.item()
            if i % 200 == 199:    # print every 20 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}')
                running_loss = 0.0

    # On ferme la barre de progression
    pbar.close()
    
    # On retourne le modèle
    return model

In [ ]:
# On créé la classe Trainer qui implémente la fonction d'entrainement du modèle
class Trainer:
    def __init__(self, model, dataloader, epochs, optimizer, criterion):
        self.model = model
        self.dataloader = dataloader
        self.epochs = epochs
        self.optimizer = optimizer
        self.criterion = criterion
    
    def train_model(self):
        train_model(self.model, self.dataloader, self.epochs, self.optimizer, self.criterion)
        return self.model

# Conclusion :

Les classes et méthodes définies dans cette parties peuvent être utilisées lors de l'entrainement de n'importe quel modèle dans le notebook `cnn_tests.ipynb`, conjointement avec la classe `Subset` du notebook `dataset_utils.ipynb`.